In [81]:
import pandas as pd
from IPython.display import display, HTML

cities = """ירושלים 352 צפת 18 אופקים 5 מגאר 4 בני ברק 267 רמלה 18 דייר חנא 5 עכו 4 תל אביב - יפו 211 גבעתיים 17 מסלול 5 פרדס חנה-כרכור 4 לא ידוע 194 גדרה 17 סח'נין 5 צור הדסה 4 אשקלון 78 שוהם 17 עלי זהב 5 קרית ביאליק 4 ראשון לציון 76 אבן יהודה 15 עמנואל 5 אורה 3 מודיעין-מכבים-רעות 63 אספר 15 קדימה-צורן 5 אלפי מנשה 3 פתח תקווה 63 גן יבנה 14 קרית מוצקין 5 ברקת 3 אשדוד 58 קרית גת 14 קרית שמונה 5 גבע בנימין 3 מגדל העמק 57 מעלה אדומים 13 שדרות 5 הר אדר 3 רמת גן 55 נהריה 13 שילה 5 כפר חושן 3 נתניה 52 קרית ים 13 שמעה 5 כפר תבור 3 בת ים 51 גני תקווה 12 אבני איתן 4 מבשרת ציון 3 חולון 51 דימונה 12 אזור 4 מעלות-תרשיחא 3 אפרת 46 זכרון יעקב 12 באר יעקב 4 מצפה רמון 3 יהוד-מונוסון 46 חדרה 12 גבעת זאב 4 נוף הגליל 3 חיפה 44 יבנה 12 זיתן 4 נס ציונה 3 אלעד 43 קרית אתא 10 יבנאל 4 נשר 3 באר שבע 43 אלון שבות 9 יובלים 4 סביון 3 רחובות 39 יקנעם עילית 9 מגאר 4 סנסנה 3 רעננה 39 כפר יונה 9 עכו 4 עופרה 3 הרצליה 38 ראש העין 9 פרדס חנה-כרכור 4 צור יצחק 3 בית שמש 37 גבעת שמואל 8 צור הדסה 4 קצרין 3 אור יהודה 32 קרית ארבע 8 קרית ביאליק 4 קרני שומרון 3 גילת 31 טירת כרמל 7 אורה 3 רנן 3 לוד 30 כוכב יעקב 7 אלפי מנשה 3 שלומי 3 קרית יערים 30 מעלה מכמש 7 ברקת 3 אור עקיבא 2 כפר סבא 29 נווה דניאל 7 גבע בנימין 3 אחיעזר 2 ענב 26 נתיבות 7 הר אדר 3 אלקנה 2 ביתר עילית 25 תל מונד 7 כפר חושן 3 אמציה 2 כפר חב"ד 25 אילת 6 כפר תבור 3 בארי 2 עפולה 25 אירוס 6 מבשרת ציון 3 בוקעאתא 2 קרית מלאכי 25 טמרה 6 מעלות-תרשיחא 3 ביצרון 2 רמת השרון 24 כוכב יאיר 6 מצפה רמון 3 בית עובד 2 טבריה 22 כרמיאל 6 נוף הגליל 3 בני עי"ש 2 מודיעין עילית 19 עומר 6 נס ציונה 3 בת חן 2 קרית אונו 19 ערד 6 נשר 3 גבע כרמל 2 הוד השרון 18 פסגות 6 סביון 3 דבוריה 2 הושעיה 2 בית גמליאל 1 כוכב השחר 1 עתניאל 1 זיקים 2 בית זית 1 כפר אחים 1 פרדסיה 1 חנתון 2 בית יצחק-שער חפר 1 כפר האורנים 1 צופית 1 חריש 2 בית נקופה 1 כפר נטר 1 קרית עקרון 1 יפעת 2 בית שקמה 1 כפר שמאי 1 רהט 1 כסיפה 2 בני ציון 1 כרם בן זמרה 1 רכסים 1 כפר אדומים 2 בר גיורא 1 להב 1 רמות השבים 1 כפר ביל"ו 2 ברקן 1 לפיד 1 רמות מאיר 1 כפר ויתקין 2 בת חפר 1 מגדל 1 שדה אילן 1 כפר תפוח 2 גבעת ברנר 1 מיתר 1 שדה ורבורג 1 להבים 2 גבעת ניל"י 1 מכמנים 1 שדה יעקב 1 מגדים 2 גיאה 1 מסעודין אל-עזאזמה 1 שדי חמד 1 מזכרת בתיה 2 גילון 1 מעגן 1 שורש 1 מירון 2 גן יאשיה 1 מעון 1 שושנת העמקים 1 מעגלים 2 גרופית 1 מעין ברוך 1 שחר 1 מצדות יהודה 2 דלתון 1 מעש 1 שכניה 1 מרכז שפירא 2 דקל 1 מצפה יריחו 1 שלומית 1 מתן 2 המעפיל 1 מצפה נטופה 1 שעל 1 נחלים 2 הרדוף 1 משמרת 1 שפרעם 1 עין שריד 2 חגור 1 נאעורה 1 שרונה 1 ציפורי 2 חולית 1 נווה ימין 1 שרשרת 1 רמת ישי 2 חוקוק 1 נווה מבטח 1 תאשור 1 רשפון 2 חירות 1 נוף איילון 1 תומר 1 שדמה 2 חמד 1 נחושה 1 תל יוסף 1 שומריה 2 חצרים 1 נטעים 1 תקוע 1 שמשית 2 חרוצים 1 ניצני עוז 1 תל יצחק 2 חרשים 1 ניר בנים 1 אבו גוש 1 חשמונאים 1 ניר ישראל 1 אביחיל 1 טייבה 1 ניר משה 1 אבני חפץ 1 טירת יהודה 1 ניר צבי 1 אום אל-פחם 1 טל שחר 1 ספיר 1 אורות 1 יד בנימין 1 עדנים 1 אחוזת ברק 1 יפיע 1 עין הנצי"ב 1 איתמר 1 ירוחם 1 עין תמר 1 אכסאל 1 ירחיב 1 עינת 1 אל סייד 1 ירקונה 1 עראבה 1 אלונים 1 יתד 1 ערוגות 1 אריאל 1 כברי 1 עתלית 1"""
cities_delim = cities.split()

In [82]:
temp = []
dict_cities = {}
for part in cities_delim:
    if part.isnumeric():
        dict_cities[' '.join(temp)] = int(part)
        temp = []
    else:
        temp.append(part)

In [83]:
cities_pretty = pd.DataFrame.from_dict(dict_cities, orient='index',columns=['pop'])
cities_pretty['city names'] = [str(i) for i in cities_pretty.index]
cities_pretty['wiki article'] = ["""https://nominatim.openstreetmap.org/search.php?q="""+str(i)+"""&format=json""" for i in cities_pretty.index]
cities_pretty['wiki'] = cities_pretty['wiki article'].apply(lambda x: '<a href="{0}">link</a>'.format(x))
display(HTML(cities_pretty.sort_values(by=['pop'],ascending=False).head().to_html(escape=False)))

,pop,city names,wiki article,wiki
ירושלים,352,ירושלים,https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json,link
בני ברק,267,בני ברק,https://nominatim.openstreetmap.org/search.php?q=בני ברק&format=json,link
תל אביב - יפו,211,תל אביב - יפו,https://nominatim.openstreetmap.org/search.php?q=תל אביב - יפו&format=json,link
לא ידוע,194,לא ידוע,https://nominatim.openstreetmap.org/search.php?q=לא ידוע&format=json,link
אשקלון,78,אשקלון,https://nominatim.openstreetmap.org/search.php?q=אשקלון&format=json,link


In [84]:
import requests
import json
import time

def get_geo(link):
    time.sleep(0.5)
    try:
        r = requests.get(link, timeout=1)
        if r.status_code  == 200:
            try:
                data = json.loads(r.content)
                if len(data[0]) > 1:
                    #display(data[0])
                    lat = data[0]['lat']
                    lon = data[0]['lon']
                    return lat, lon, True
                else:
                    return 0, 0, False
            except IndexError:
                return 0, 0, False
            except TypeError:
                return 0, 0, False
        else:
            return 0, 0, False
    except OSError:
        print("Failed to load {}".format(link))
        return 0, 0, False


In [85]:
print(cities_pretty['wiki article'][0])
get_geo(cities_pretty['wiki article'][0])

https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json


('31.778345', '35.2250786', True)

In [86]:
lats = []
lons = []
valids = []

import ipywidgets as widgets

bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(cities_pretty['wiki article']),
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)


display(bar)

for n, i in enumerate(cities_pretty['wiki article']):
    lat, lon, valid_geo = get_geo(i)
    lats.append(lat)
    lons.append(lon)
    valids.append(valid_geo)
    bar.value = (n+1)




IntProgress(value=0, description='Loading:', max=272)

Failed to load https://nominatim.openstreetmap.org/search.php?q=תומר&format=json


In [87]:
cities_pretty['lat'] = lats
cities_pretty['lon'] = lons
cities_pretty['valid GEO'] = valids
cities_pretty['text'] = 'Ammount of sick at: ' + cities_pretty['city names'] +' '+ cities_pretty['pop'].astype(str)

In [88]:
display(HTML(cities_pretty.sort_values(by=['pop'],ascending=False).head().to_html(escape=False)))
cities_pretty.sort_values(by=['pop'],ascending=False).to_csv('CompleteCities.csv')

,pop,city names,wiki article,wiki,lat,lon,valid GEO,text
ירושלים,352,ירושלים,https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json,link,31.778345,35.2250786,True,Ammount of sick at: ירושלים 352
בני ברק,267,בני ברק,https://nominatim.openstreetmap.org/search.php?q=בני ברק&format=json,link,32.0873899,34.8324376,True,Ammount of sick at: בני ברק 267
תל אביב - יפו,211,תל אביב - יפו,https://nominatim.openstreetmap.org/search.php?q=תל אביב - יפו&format=json,link,32.0804808,34.7805274,True,Ammount of sick at: תל אביב - יפו 211
לא ידוע,194,לא ידוע,https://nominatim.openstreetmap.org/search.php?q=לא ידוע&format=json,link,0,0,False,Ammount of sick at: לא ידוע 194
אשקלון,78,אשקלון,https://nominatim.openstreetmap.org/search.php?q=אשקלון&format=json,link,31.6644874,34.5730157,True,Ammount of sick at: אשקלון 78


In [89]:
import plotly.graph_objects as go


fig = go.Figure(data=go.Scattergeo(
        lon = cities_pretty['lon'],
        lat = cities_pretty['lat'],
        text = cities_pretty['text'],
        mode = 'markers',
        marker_color = cities_pretty['pop'],
        marker_size = cities_pretty['pop'],
        ))

fig.update_layout(
        title = 'Israeli Patients',
        geo_scope='world',
    )

fig.show()

In [110]:
import plotly.express as px
df = cities_pretty[cities_pretty['valid GEO']==True]
fig = px.scatter_geo(df,
                     locationmode = 'country names',
                    lon = df['lon'],
                    lat = df['lat'],
                     hover_name="text", # column added to hover information
                     size="pop", # size of markers
                     projection = 'natural earth',
                    #projection_scale= 0.1,
                    center = {'lat': 31.778345, 'lon':35.2250786})
fig.update_geos(
    showcountries=True, countrycolor="RebeccaPurple")

fig.show()